## Practical 1: word2vec
<p>Oxford CS - Deep NLP 2017<br>
https://www.cs.ox.ac.uk/teaching/courses/2016-2017/dl/</p>
<p>[Yannis Assael, Brendan Shillingford, Chris Dyer]</p>

This practical is presented as an IPython Notebook, with the code written for recent versions of **Python 3**. The code in this practical will not work with Python 2 unless you modify it. If you are using your own Python installation, ensure you have a setup identical to that described in the installation shell script (which is intended for use with the department lab machines). We will be unable to support installation on personal machines due to time constraints, so please use the lab machines and the setup script if you are unfamiliar with how to install Anaconda.

To execute a notebook cell, press `shift-enter`. The return value of the last command will be displayed, if it is not `None`.

Potentially useful library documentation, references, and resources:

* IPython notebooks: <https://ipython.org/ipython-doc/3/notebook/notebook.html#introduction>
* Numpy numerical array library: <https://docs.scipy.org/doc/>
* Gensim's word2vec: <https://radimrehurek.com/gensim/models/word2vec.html>
* Bokeh interactive plots: <http://bokeh.pydata.org/en/latest/> (we provide plotting code here, but click the thumbnails for more examples to copy-paste)
* scikit-learn ML library (aka `sklearn`): <http://scikit-learn.org/stable/documentation.html>
* nltk NLP toolkit: <http://www.nltk.org/>
* tutorial for processing xml in python using `lxml`: <http://lxml.de/tutorial.html> (we did this for you below, but in case you need it in the future)

In [1]:
import numpy as np
import os
from random import shuffle
import re

In [3]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

### Part 0: Download the TED dataset

In [6]:
import urllib
import zipfile
import lxml.etree

In [7]:
# Download the dataset if it's not already there: this may take a minute as it is 75MB
if not os.path.isfile('ted_en-20160408.zip'):
    urllib.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")

In [8]:
# For now, we're only interested in the subtitle text, so let's extract that from the XML:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))
del doc

### Part 1: Preprocessing

In this part, we attempt to clean up the raw subtitles a bit, so that we get only sentences. The following substring shows examples of what we're trying to get rid of. Since it's hard to define precisely what we want to get rid of, we'll just use some simple heuristics.

In [9]:
i = input_text.find("Hyowon Gweon: See this?")
input_text[i-20:i+150]

u' baby does.\n(Video) Hyowon Gweon: See this? (Ball squeaks) Did you see that? (Ball squeaks) Cool. See this one? (Ball squeaks) Wow.\nLaura Schulz: Told you. (Laughs)\n(Vide'

Let's start by removing all parenthesized strings using a regex:

In [10]:
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)

We can verify the same location in the text is now clean as follows. We won't worry about the irregular spaces since we'll later split the text into sentences and tokenize it anyway.

In [11]:
i = input_text_noparens.find("Hyowon Gweon: See this?")
input_text_noparens[i-20:i+150]

u"hat the baby does.\n Hyowon Gweon: See this?  Did you see that?  Cool. See this one?  Wow.\nLaura Schulz: Told you. \n HG: See this one?  Hey Clara, this one's for you. You "

Now, let's attempt to remove speakers' names that occur at the beginning of a line, by deleting pieces of the form "`<up to 20 characters>:`", as shown in this example. Of course, this is an imperfect heuristic. 

In [12]:
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

# Uncomment if you need to save some RAM: these strings are about 50MB.
# del input_text, input_text_noparens

# Let's view the first few:
sentences_strings_ted[:5]

[u"Here are two reasons companies fail: they only do more of the same, or they only do what's new",
 u'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation',
 u' Both are necessary, but it can be too much of a good thing',
 u'Consider Facit',
 u" I'm actually old enough to remember them"]

Now that we have sentences, we're ready to tokenize each of them into words. This tokenization is imperfect, of course. For instance, how many tokens is "can't", and where/how do we split it? We'll take the simplest naive approach of splitting on spaces. Before splitting, we remove non-alphanumeric characters, such as punctuation. You may want to consider the following question: why do we replace these characters with spaces rather than deleting them? Think of a case where this yields a different answer.

In [13]:
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

Two sample processed sentences:

In [14]:
len(sentences_ted)

266694

In [15]:
print(sentences_ted[0])
print(sentences_ted[1])

[u'here', u'are', u'two', u'reasons', u'companies', u'fail', u'they', u'only', u'do', u'more', u'of', u'the', u'same', u'or', u'they', u'only', u'do', u'what', u's', u'new']
[u'to', u'me', u'the', u'real', u'real', u'solution', u'to', u'quality', u'growth', u'is', u'figuring', u'out', u'the', u'balance', u'between', u'two', u'activities', u'exploration', u'and', u'exploitation']


### Part 2: Word Frequencies

If you store the counts of the top 1000 words in a list called `counts_ted_top1000`, the code below will plot the histogram requested in the writeup.

In [20]:
from nltk import FreqDist

word_list = [word for sentence in sentences_ted for word in sentence]
distribution = FreqDist(word_list)

In [21]:
top1000 = distribution.most_common(1000)
counts_ted_top1000 = [count for word, count in top1000]
words_ted_top1000 = [word for word, count in top1000]

Plot distribution of top-1000 words

In [22]:
hist, edges = np.histogram(counts_ted_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

### Part 3: Train Word2Vec

In [19]:
from gensim.models import Word2Vec

In [24]:
model_ted = Word2Vec(sentences_ted, size=100, window=5, min_count=10, workers=4)

### Part 4: Ted Learnt Representations

Finding similar words: (see gensim docs for more functionality of `most_similar`)

In [25]:
model_ted.most_similar("man")

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[(u'woman', 0.8424171209335327),
 (u'guy', 0.8048239946365356),
 (u'lady', 0.7494685649871826),
 (u'girl', 0.7398644089698792),
 (u'boy', 0.7373195886611938),
 (u'gentleman', 0.7252902984619141),
 (u'soldier', 0.7003352642059326),
 (u'kid', 0.6968482136726379),
 (u'david', 0.6589904427528381),
 (u'person', 0.6465502977371216)]

In [26]:
model_ted.most_similar("computer")

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[(u'machine', 0.7426152229309082),
 (u'software', 0.7257680892944336),
 (u'device', 0.6978968977928162),
 (u'robot', 0.6865769624710083),
 (u'simulation', 0.6754375696182251),
 (u'program', 0.6595192551612854),
 (u'camera', 0.6548471450805664),
 (u'interface', 0.6512086987495422),
 (u'chip', 0.6417384147644043),
 (u'3d', 0.6354435682296753)]

In [27]:
model_ted.most_similar("the")

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[(u'manhattan', 0.4065490961074829),
 (u'shining', 0.3978366255760193),
 (u'exact', 0.3618057370185852),
 (u'its', 0.3610724210739136),
 (u'upper', 0.35671842098236084),
 (u'reflected', 0.35611021518707275),
 (u'tagging', 0.3557807505130768),
 (u'antarctica', 0.3513306677341461),
 (u'academia', 0.34912610054016113),
 (u'monterey', 0.34778308868408203)]

In [28]:
model_ted.most_similar("ted")

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[(u'conference', 0.742803692817688),
 (u'tedwomen', 0.6079268455505371),
 (u'chris', 0.5894644260406494),
 (u'mit', 0.5735347270965576),
 (u'tonight', 0.5513068437576294),
 (u'professor', 0.5366588830947876),
 (u'richard', 0.5285269618034363),
 (u'steve', 0.5260133743286133),
 (u'john', 0.5250959992408752),
 (u'dan', 0.5139486789703369)]

#### t-SNE visualization
To use the t-SNE code below, first put a list of the top 1000 words (as strings) into a variable `words_top_ted`. The following code gets the corresponding vectors from the model, assuming it's called `model_ted`:

In [30]:
# This assumes words_top_ted is a list of strings, the top 1000 words
words_top_vec_ted = model_ted[words_ted_top1000]

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [32]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(words_top_vec_ted)

In [34]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words_ted_top1000))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

### Part 5: Wiki Learnt Representations

Download dataset

In [17]:
if not os.path.isfile('wikitext-103-raw-v1.zip'):
    urllib.urlretrieve("https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip", filename="wikitext-103-raw-v1.zip")

In [18]:
with zipfile.ZipFile('wikitext-103-raw-v1.zip', 'r') as z:
    input_text = unicode(z.open('wikitext-103-raw/wiki.train.raw', 'r').read(), 'utf-8') # Thanks Robert Bastian

Preprocess sentences (note that it's important to remove small sentences for performance)

In [35]:
sentences_wiki = []
for line in input_text.split('\n'):
    s = [x for x in line.split('.') if x and len(x.split()) >= 5]
    sentences_wiki.extend(s)
    
for s_i in range(len(sentences_wiki)):
    sentences_wiki[s_i] = re.sub("[^a-z]", " ", sentences_wiki[s_i].lower())
    sentences_wiki[s_i] = re.sub(r'\([^)]*\)', '', sentences_wiki[s_i])
del input_text

In [36]:
# sample 1/5 of the data
shuffle(sentences_wiki)
print(len(sentences_wiki))
sentences_wiki = sentences_wiki[:int(len(sentences_wiki)/5)]
print(len(sentences_wiki))

4267112
853422


In [45]:
print sentences_wiki[:2]

[u' in persona   they are based on japanese deities   persona   will use characters inspired by popular fictional and historical outlaws ', u' the signing of the agreement starts the clock for numerous deadlines that must be met in an effort to improve the department  s handling of use     of     force incidents ']


In [46]:
tokenised_sentences_wiki = []
for sent_str in sentences_wiki:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    tokenised_sentences_wiki.append(tokens)

Now, repeat all the same steps that you performed above. You should be able to reuse essentially all the code.

In [49]:
wiki_word_list = [word for sentence in tokenised_sentences_wiki for word in sentence]
distribution = FreqDist(wiki_word_list)
top1000 = distribution.most_common(1000)
counts_wiki_top1000 = [count for word, count in top1000]
words_wiki_top1000 = [word for word, count in top1000]

In [50]:
hist, edges = np.histogram(counts_wiki_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

In [51]:
words_wiki_top1000[:5]

[u'the', u'of', u'and', u'in', u'to']

In [55]:
model_wiki = Word2Vec(tokenised_sentences_wiki, size=100, window=5, min_count=10, workers=4)

In [61]:
model_wiki.most_similar("man")

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[(u'woman', 0.7282618284225464),
 (u'girl', 0.6200228333473206),
 (u'boy', 0.6115930080413818),
 (u'person', 0.5995253920555115),
 (u'dog', 0.5883893966674805),
 (u'gentleman', 0.5523183345794678),
 (u'creature', 0.5517150163650513),
 (u'men', 0.5507960319519043),
 (u'soldier', 0.5464372634887695),
 (u'someone', 0.5208908319473267)]

In [57]:
model_wiki.most_similar("computer")

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[(u'software', 0.7671365737915039),
 (u'handheld', 0.7092382907867432),
 (u'user', 0.6960841417312622),
 (u'hardware', 0.6773449182510376),
 (u'controller', 0.6767948865890503),
 (u'computing', 0.675836443901062),
 (u'computers', 0.6741385459899902),
 (u'interactive', 0.6654195189476013),
 (u'technology', 0.6643165349960327),
 (u'interface', 0.6608816981315613)]

In [58]:
model_wiki.most_similar("the")

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[(u'its', 0.5616340637207031),
 (u'this', 0.5445227026939392),
 (u'a', 0.5007539391517639),
 (u'each', 0.48358261585235596),
 (u'another', 0.4739660620689392),
 (u'itself', 0.44650253653526306),
 (u'their', 0.44057753682136536),
 (u'any', 0.43795984983444214),
 (u'entire', 0.4226207137107849),
 (u'one', 0.39856699109077454)]

In [62]:
model_wiki.most_similar("ted")

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[(u'josh', 0.8865985870361328),
 (u'terry', 0.8843636512756348),
 (u'doug', 0.8802260756492615),
 (u'colin', 0.8785141706466675),
 (u'glenn', 0.876736044883728),
 (u'greg', 0.875827968120575),
 (u'gary', 0.8749195337295532),
 (u'fred', 0.8716672658920288),
 (u'casey', 0.8710375428199768),
 (u'larry', 0.8695237636566162)]

#### t-SNE visualization

In [59]:
# This assumes words_top_wiki is a list of strings, the top 1000 words
words_top_vec_wiki = model_wiki[words_wiki_top1000]

tsne = TSNE(n_components=2, random_state=0)
words_top_wiki_tsne = tsne.fit_transform(words_top_vec_wiki)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [63]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_wiki_tsne[:,0],
                                    x2=words_top_wiki_tsne[:,1],
                                    names=words_wiki_top1000))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)